# Distributions

In [1]:
import pandas as pd

from msdm.core.distributions import DiscreteFactorTable as Pr
from msdm.core.assignment import AssignmentMap as Dict

def toDF(p):
    df = pd.DataFrame(p.support)
    df['prob'] = p.probs
    df['scores'] = p.scores
    return df

# Conjunction / Products
The conjunction of two distributions $p_{0}(A, B)$ and $p_1(B, C)$ is $p_2(A, B, C)$ where:
$$
p_2(A, B, C) \propto p_0(A, B)p_1(B, C)
$$

In [2]:
# Conjunction (independent variables)
pA = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[.5, .5])
p = pA & pB
toDF(p)

,a,b,prob,scores
0,0,0,0.45,-0.798508
1,0,1,0.45,-0.798508
2,1,0,0.05,-2.995732
3,1,1,0.05,-2.995732


In [3]:
# Conjunction
pA1 = Pr([{'a': 0}, {'a': 1}, {'a': 2}], probs=[.7, .2, .1])
pA2 = Pr([{'a': 1}, {'a': 2}, {'a': 3}], probs=[.5, .4, .1])
p = pA1 & pA2
toDF(p.normalize())

,a,prob,scores
0,1,0.714286,-0.336472
1,2,0.285714,-1.252763


In [4]:
# Conjunction (dependent variables)
pAB = Pr([{'a': 0, 'b': 0}, {'a': 1, 'b': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[1/3, 2/3])
p = pAB & pB
toDF(p.normalize())

,a,b,prob,scores
0,0,0,0.818182,-0.200671
1,1,1,0.181818,-1.704748


In [5]:
# Conjunction (dependent variables)
pAB = Pr([{'a': 0, 'b': 0}, {'a': 1, 'b': 1}], probs=[.9, .1])
pBC = Pr([{'b': 0, 'c': 0}, {'b': 1, 'c': 0}, {'b': 1, 'c': 1}], probs=[1/3, 1/3, 1/3])
p = pAB & pBC
toDF(p.normalize())

,a,b,c,prob,scores
0,0,0,0,0.818182,-0.200671
1,1,1,0,0.090909,-2.397895
2,1,1,1,0.090909,-2.397895


# Disjunction / Mixtures

The disjunction of two distributions $p_0(A, B)$ and $p_1(B, C)$ is $p_2(A, B, C)$ where:
$$
p_2(A, B, C) \propto \alpha_0 p_0(A, B) + \alpha_1 p_1(B, C)
$$
where $\alpha_0$ and $\alpha_1$ are mixing weights.

In [6]:
# Mixture of Distributions
pA1 = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pA2 = Pr([{'a': 1}, {'a': 2}], probs=[.5, .5])
p = pA1*.1 | pA2*.9
toDF(p.normalize())

,a,prob,scores
0,1,0.46,-0.776529
1,0,0.09,-2.407946
2,2,0.45,-0.798508


In [7]:
# marginalization
pA = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[.1, .9])
pAB = pA & pB
pAeqB = pAB[lambda r: "a == b" if r['a'] == r['b'] else "a != b"]
toDF(pAeqB)

,0,prob,scores
0,a == b,0.18,-1.714798
1,a != b,0.82,-0.198451


In [8]:
# new variable assignment
pA = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pAb = pA[lambda r: {**r, 'b': 1}]
toDF(pAb)

,a,b,prob,scores
0,0,1,0.9,-0.105361
1,1,1,0.1,-2.302585


In [9]:
# Conditional Probability Example

# p(A | B = 1)
pAB1 = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pAB1 = pAB1[lambda r: {**r, "b": 1}] #sets b=1 to all elements

# p(A | B = 2)
pAB2 = Pr([{'a': 0}, {'a': 1}], probs=[.2, .8])
pAB2 = pAB2[lambda r: {**r, "b": 2}] #sets b=2 to all elements

# p(B)
pB = Pr([{'b': 1}, {'b': 2}], probs=[.5, .5])

# p(A, B) = p(A | B = 1)p(B = 1) + p(A | B = 2)p(B = 2)
pAmidB = (pAB1 | pAB2)
pAB = pAmidB & pB
toDF(pAB.normalize())

,a,b,prob,scores
0,0,1,0.45,-0.798508
1,0,2,0.10,-2.302585
2,1,2,0.40,-0.916291
3,1,1,0.05,-2.995732


In [10]:
# expressions
pA = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[.1, .9])
pAB = pA & pB
pAeqB = pAB["a + b"]
toDF(pAeqB)

,0,prob,scores
0,0,0.09,-2.407946
1,1,0.82,-0.198451
2,2,0.09,-2.407946
